# COMP47590: Advanced Machine Learning
# Assignment 1: The Super Learner
# Student No: 16213128

## Import Packages 

In [1]:
import os
import subprocess
import io 
import copy
from IPython.display import display, HTML, Image
from operator import itemgetter


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
from random import randint


from sklearn.tree import export_graphviz

from sklearn import tree
from sklearn import ensemble
from sklearn import linear_model
from sklearn import neighbors
from sklearn import neural_network
from sklearn import svm

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn import metrics


%matplotlib inline
#%qtconsole

## Define Super Learner Classifier

The *Super Learner* is a heterogeneous stacked ensemble classifier. This is a classification model that uses a set of base classifiers of different types, the outputs of which are then combined in another classifier at the stacked layer. The Super Learner was described in [(van der Laan et al, 2007)](https://pdfs.semanticscholar.org/19e9/c732082706f39d2ba12845851309714db135.pdf) but the stacked ensemble idea has been around for a long time. 

Figure 1 shows a flow diagram of the Super Learner process (this is from (van der Laan et al, 2007) and the process is also described in the COMP47590 lecture "[COMP47590 2017-2018 L04 Supervised Learning Ensembles 3](https://www.dropbox.com/s/1ksx94nxtuyn4l8/COMP47590%202017-2018%20L04%20Supervised%20Learning%20Ensembles%203.pdf?raw=1)"). The base classifiers are trained and their outputs are combined along with the training dataset labels into a training set for the stack layer classifier. To avoid overfitting the generation of the stacked layer training set uses a k-fold cross validation process (described as V-fold in Figure 1). To further add variety to the base estimators a bootstrapping selection (as is used in the bagging ensemble approach).
 
[Super Learner Process Flow]

Figure 1: A flow diagram for the Super Learner


### Task1:  Define the SuperLearnerClassifier Class

In [2]:
# Create a new classifier which is based on the sckit-learn BaseEstimator and ClassifierMixin classes
class SuperLearnerClassifier(BaseEstimator, ClassifierMixin):
    
    
    # Constructor for the classifier object, proba_flag = False(the output from base layer is label output)
    #  proba_flag = True(the output from base layer is probability output)
    def __init__(self, base_models, stack_key = "DT", proba_flag = False, original_data_flag = False, param_grid = {}):
        """
        Setup a SuperLearner classifier .
        """ 
        self.base_models = base_models
        
        self.stack_layer_models = {
            'LR':linear_model.LogisticRegression(penalty='l2'), # Logistic Regression Classifier 
            'RF':ensemble.RandomForestClassifier(n_estimators=8),  # Random Forest Classifier 
            'SVM':svm.SVC(kernel='rbf', probability=True),  # SVM Classifier
            'DT':tree.DecisionTreeClassifier(criterion="entropy"),  # Decision Tree Classifier
            'KNN':neighbors.KNeighborsClassifier(), # KNN Classifier
            'NN':neural_network.MLPClassifier(hidden_layer_sizes=(300, 100)),  # Neural Network Classifier
            'GB':ensemble.GradientBoostingClassifier(n_estimators=10), # GradientBoostingClassifier 
        }
        
        self.stack_key = stack_key
        
        self.n_folds = 5
        self.proba_flag = proba_flag
        self.original_data_flag = original_data_flag
        self.label_counts = {}
        self.preds_list = [] # for Task9 Diversity Tests Report
        self.param_grid = param_grid
        self.model_tuned_params_list = dict()
        self.best_score_list = dict()
        self.model_objects = dict()
        
    
    # suppose training data 5000 rows
    def get_k_fold_cv_training_data(self, clf, train_x, train_y):
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=2017)
        
        # calculate the numbers for different class
        if len(self.label_counts) == 0:
            for x in train_y:                      
                if  x in self.label_counts:
                    self.label_counts[x] += 1
                else:
                    self.label_counts[x] = 1
            #print(self.label_counts)
        
        # The shape attribute for numpy arrays returns the dimensions of the array. 
        # If Y has n rows and m columns, then Y.shape is (n,m) . So Y.shape[0] is n .
        # zeros: Depending on the given shape and type(shape[, dtype, order]) returns an array with a new element of all 0
        if self.proba_flag == False:
            su_train_y = np.empty((train_x.shape[0],))  # train set  5000 row * 1
        elif self.proba_flag == True:
            su_train_y = np.empty((train_x.shape[0], len(self.label_counts)))  # train set  5000 row * label_count
        
        
        # kf.split() Generate indices to split data into training(ndarray) and test(ndarray) set.
        for i,(train_index, test_index) in enumerate(kf.split(train_x)): # train_x: 5000 rows * 10 columns(features) 
          
            kf_train_x = train_x[train_index]   # 4000 rows * 10 features; 4000 instances for each fold
            kf_train_y = train_y[train_index]   # 4000 rows * 1 label;
            kf_test_x = train_x[test_index]     # 1000 rows * 10 features;
        
            clf.fit(kf_train_x, kf_train_y)
            
            if self.proba_flag == False:
                su_train_y[test_index] = clf.predict(kf_test_x) # 1000 row * 1 predicted label ====> will be 5000 row * 1 label after 5 folds
            elif self.proba_flag == True:
                su_train_y[test_index] = clf.predict_proba(kf_test_x)
                
        return su_train_y

    
    # suppose test data 1200 rows
    def get_k_fold_cv_test_data(self, clf, test_x):
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=2017)
        
        if self.proba_flag == False:
            su_test_y = np.empty((test_x.shape[0],))  # train set  5000 row * 1
        elif self.proba_flag == True:
            su_test_y = np.empty((test_x.shape[0],len(self.label_counts)))  # train set  5000 row * label_count
        
        # kf.split() Generate indices to split data into training(ndarray) and test(ndarray) set.
        for i,(train_index, test_index) in enumerate(kf.split(test_x)): # test_x: 5000 rows * 10 columns(features)
            
            kf_test_x = test_x[test_index]     # 1000 rows * 10 features;
            
            if self.proba_flag == False:
                su_test_y[test_index] = clf.predict(kf_test_x) # 1000 row * 1 predicted label ====> will be 5000 row * 1 label after 5 folds
            elif self.proba_flag == True:
                su_test_y[test_index] = clf.predict_proba(kf_test_x)
        
        '''
        for i in range(self.n_folds):
            su_test_skf_y[i,:] = clf.predict(test_x)  #  1200 row * 1 label test set for each fold
        
        # 2 demension array exchange its rows and columns
        su_test_skf_y_temp = list(list(i) for i in zip(*su_test_skf_y))
        
        # get majority voting from su_test_skf_y(from 5 label results to choose the most one)
        from collections import Counter
        for i, value in enumerate(su_test_skf_y_temp):
            element_counts = Counter(value)
            top_one = element_counts.most_common(1)
            su_test_y[i] = top_one[0][0]
        '''
        
        return su_test_y
            
          

    # The fit function to train a classifier
    def fit(self, X, y):
        stack_training_y = None
        count = 0
        
        model_objects = {}
        for index in self.base_models:
            model_objects[index] = self.base_models[index]
            new_training_labels = self.get_k_fold_cv_training_data(model_objects[index], X, y)
            
            # for Task9 Diversity Tests Report
            self.preds_list.append(new_training_labels) 
            if count == 0:
                stack_training_y = new_training_labels
            else:
                stack_training_y = np.c_[stack_training_y, np.array(new_training_labels)]
            count = count + 1
            
            # task7 GridSearch:
            if len(self.param_grid) != 0:
                my_tuned_model = GridSearchCV(model_objects[index], self.param_grid[index], cv=3, verbose = 2)
                my_tuned_model.fit(X, y)
                model_objects[index] = my_tuned_model
                # Print details
                print(index)
                print("Best parameters set found on development set:")
                print(my_tuned_model.best_params_)
                self.model_tuned_params_list["Tuned "+index+"===="] = my_tuned_model.best_params_.copy()
                print(my_tuned_model.best_score_)
                self.best_score_list[index] = my_tuned_model.best_score_.copy()
                print(my_tuned_model.cv_results_)
                print("**************************************************************")
            
        # for keeping fit() and predict() have the same estimator object
        self.model_objects = model_objects.copy()
            
        # self.stack_training = list(list(i) for i in zip(*stack_training_y)) # n rows * 6 labels columns
        #display(list(stack_training_y)) # n rows * 6 labels columns
        if(self.original_data_flag == True):
            stack_training_y = np.c_[np.array(X), stack_training_y]
            
        self.layer_model = copy.deepcopy(self.stack_layer_models[self.stack_key].fit(stack_training_y, y))                                               
        return self
    
    

    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):
        stack_test_y = None
        count = 0
        
        for index in self.model_objects:
            new_test_labels = self.get_k_fold_cv_test_data(self.model_objects[index], X)
            if count == 0:
                stack_test_y = new_test_labels
            else:
                stack_test_y = np.c_[stack_test_y, np.array(new_test_labels)]
            count = count + 1
        
        #display(list(stack_training_y)) # n rows * 6 labels columns
        if(self.original_data_flag == True):
            stack_test_y = np.c_[np.array(X), stack_test_y]
            
        #stack_test = list(list(i) for i in zip(*stack_test_y)) # m rows * 6 labels columns
        predictions = self.layer_model.predict(stack_test_y)
        
        return predictions
    
    
    
    # The predict function to make a set of predictions for a set of query instances
    def predict_proba(self, X):
        stack_test_y = None
        count = 0
        
        for index in self.model_objects:
            new_test_labels = self.get_k_fold_cv_test_data(self.model_objects[index], X)
            if count == 0:
                stack_test_y = new_test_labels
            else:
                stack_test_y = np.c_[stack_test_y, np.array(new_test_labels)]
            count = count + 1
            
        #display(list(stack_training_y)) # n rows * 6 labels columns
        if(self.original_data_flag == True):
            stack_test_y = np.c_[np.array(X),stack_test_y]
            
        #stack_test = list(list(i) for i in zip(*stack_test_y)) # m rows * 6 labels columns
        predictions = self.layer_model.predict_proba(stack_test_y)
        
        return predictions


Perform a simple test using the SuperLearnClassifier on the Iris dataset

In [3]:
base_model_list = {
            'LR':linear_model.LogisticRegression(penalty='l2'), # Logistic Regression Classifier 
            'RF':ensemble.RandomForestClassifier(n_estimators=8),  # Random Forest Classifier 
            'SVM':svm.SVC(kernel='rbf', probability=True),  # SVM Classifier
            'DT':tree.DecisionTreeClassifier(criterion="entropy"),  # Decision Tree Classifier
            'KNN':neighbors.KNeighborsClassifier(), # KNN Classifier
            'NN':neural_network.MLPClassifier(hidden_layer_sizes=(300, 100)),  # Neural Network Classifier
            'GB':ensemble.GradientBoostingClassifier(n_estimators=10), # GradientBoostingClassifier 
        }


In [4]:
from sklearn.datasets import load_iris
clf = SuperLearnerClassifier(base_model_list, stack_key='DT', proba_flag=True, original_data_flag = False)
iris = load_iris()
#clf.get_k_fold_cv_training_data(tree.DecisionTreeClassifier(criterion="entropy"), iris.data, iris.target)
# X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0, train_size = 0.7)

#clf.fit(iris.data, iris.target)

#display(iris.data)
scores = cross_val_score(clf, iris.data, iris.target, cv=3)
print(scores)


C:\ProgramData\Anaconda3\envs\test\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\envs\test\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\envs\test\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[1.         0.92156863 0.97916667]


## Load & Partition Data

### Setup - IMPORTANT

Take only a sample of the dataset for fast testing

In [5]:
data_sampling_rate = 0.05

Setup the number of folds for all grid searches (should be 5 - 10)

In [6]:
cv_folds = 2

### Load Dataset

Load the dataset and explore it.

Here I use test.csv for the reason that the train.csv is to some extend, my machine cannot process it and spend too much time. Actually the more suitable way is that I should use fashion-mnist_train.csv rather than fashion-mnist_test.csv

In [7]:
dataset = pd.read_csv('fashion-mnist_test.csv') # better way: dataset = pd.read_csv('fashion-mnist_train.csv')
dataset = dataset.sample(frac=data_sampling_rate) #take a sample from the dataset so everyhting runs smoothly
num_classes = 10
classes = {0: "T-shirt/top", 1:"Trouser", 2: "Pullover", 3:"Dress", 4:"Coat", 5:"Sandal", 6:"Shirt", 7:"Sneaker", 8:"Bag", 9:"Ankle boot"}
display(dataset.head())

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
1634,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8690,2,0,0,0,0,0,0,0,0,0,...,54,41,7,0,15,18,0,0,0,0
5111,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9039,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8030,6,0,0,0,0,0,63,41,1,55,...,83,119,72,0,0,0,0,0,0,0


### Pre-process & Partition Data

Perform data pre-processing and manipulation as required

In [8]:
# Isolate the descriptive features we are interested in
X = dataset[dataset.columns[1:]]
Y = np.array(dataset["label"])
# Normalise the data
#X = X/255
#Split the data into a training set, a vaidation set, and a test set
X_train_plus_valid, X_test, y_train_plus_valid, y_test  = train_test_split(X, Y, random_state=0, train_size = 0.7)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_plus_valid, y_train_plus_valid, random_state=0, train_size = 0.5/0.7)

X_train = X_train.values
X_valid = X_valid.values
X_test = X_test.values

# type(y_train)

C:\ProgramData\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


## Train and Evaluate a Simple Model

Train a Super Learner Classifier using the prepared dataset

Evaluate the trained classifier 


In [9]:
def evaluator(clf, X, y):
    # Make a set of predictions for the Dataset
    y_pred = clf.predict(X)

    # Print performance details
    accuracy = metrics.accuracy_score(y, y_pred) # , normalize=True, sample_weight=None
    print("Accuracy: " +  str(accuracy))
    print(metrics.classification_report(y, y_pred))

    # Print confusion matrix
    # print(metrics.confusion_matrix(y_train, y_pred))

    # Print nicer homemade confusion matrix
    print("Confusion Matrix")
    pd.crosstab(np.array(y), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)
    
    return

Assess the performance of the super_learner on the training set

In [10]:
my_super_learner = SuperLearnerClassifier(base_model_list, stack_key='DT', proba_flag=True, original_data_flag = False)
my_super_learner.fit(X_train, y_train)

SuperLearnerClassifier(base_models={'LR': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'RF': RandomFor...       presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)},
            original_data_flag=False, param_grid={}, proba_flag=True,
            stack_key='DT')

In [12]:
evaluator(my_super_learner, X_train, y_train)

Accuracy: 0.844
             precision    recall  f1-score   support

          0       0.74      0.74      0.74        23
          1       1.00      1.00      1.00        22
          2       0.71      0.65      0.68        26
          3       1.00      0.55      0.71        22
          4       0.91      1.00      0.95        20
          5       1.00      0.85      0.92        20
          6       0.38      0.58      0.46        26
          7       1.00      1.00      1.00        23
          8       1.00      1.00      1.00        35
          9       1.00      1.00      1.00        33

avg / total       0.87      0.84      0.85       250

Confusion Matrix


Assess the performance of the super_learner on the validation dataset

In [11]:
evaluator(my_super_learner, X_valid, y_valid)

Accuracy: 0.72
             precision    recall  f1-score   support

          0       0.83      0.83      0.83         6
          1       1.00      0.82      0.90        17
          2       0.54      0.70      0.61        10
          3       0.65      0.92      0.76        12
          4       0.57      0.44      0.50         9
          5       0.57      0.67      0.62         6
          6       0.80      0.33      0.47        12
          7       0.82      0.69      0.75        13
          8       0.75      1.00      0.86         6
          9       0.67      0.89      0.76         9

avg / total       0.74      0.72      0.71       100

Confusion Matrix


Assess the performance of the super_learner on the test dataset

In [12]:
evaluator(my_super_learner, X_test, y_test)

Accuracy: 0.6866666666666666
             precision    recall  f1-score   support

          0       0.86      0.78      0.82        23
          1       0.92      0.85      0.88        13
          2       0.43      0.55      0.48        11
          3       0.79      0.94      0.86        16
          4       0.40      0.31      0.35        13
          5       0.67      0.71      0.69        14
          6       0.12      0.08      0.10        13
          7       0.78      0.78      0.78         9
          8       0.62      0.88      0.73        17
          9       0.89      0.76      0.82        21

avg / total       0.68      0.69      0.68       150

Confusion Matrix


## Cross Validation Experiment (Task 2)

Perfrom a 10-fold cross validation experiment to evaluate the performance of the SuperLearnerClassifier

In [13]:
scores = cross_val_score(my_super_learner, X_train, y_train, cv=3)
print(scores)

[0.67045455 0.68292683 0.6375    ]


## Comparing the Performance of Different Stack Layer Approaches (Task 5)

Compare the performance of the ensemble when a label based stack layer training set and a probability based stack layer training set is used.

In [14]:
data_sampling_rate = 0.05
dataset = pd.read_csv('fashion-mnist_test.csv')
dataset = dataset.sample(frac=data_sampling_rate) #take a sample from the dataset so everyhting runs smoothly
num_classes = 10
classes = {0: "T-shirt/top", 1:"Trouser", 2: "Pullover", 3:"Dress", 4:"Coat", 5:"Sandal", 6:"Shirt", 7:"Sneaker", 8:"Bag", 9:"Ankle boot"}

X = dataset[dataset.columns[1:]]
Y = np.array(dataset["label"])

X_train, X_test, y_train, y_test \
    = train_test_split(X, Y, random_state=0, \
                                    train_size = 0.7)
X_train = X_train.values
X_test = X_test.values


C:\ProgramData\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [15]:
#  a label based stack layer training set is used and stack layer classifier is DT
my_super_learner = SuperLearnerClassifier(base_model_list, stack_key='DT', proba_flag=False, original_data_flag = False)
my_super_learner.fit(X_train, y_train)

SuperLearnerClassifier(base_models={'LR': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'RF': RandomFor...       presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)},
            original_data_flag=False, param_grid={}, proba_flag=False,
            stack_key='DT')

In [16]:
evaluator(my_super_learner, X_test, y_test)

Accuracy: 0.6066666666666667
             precision    recall  f1-score   support

          0       0.36      0.33      0.34        15
          1       1.00      0.94      0.97        18
          2       0.56      0.48      0.51        21
          3       0.53      0.73      0.62        11
          4       0.29      0.24      0.26        17
          5       0.73      0.92      0.81        12
          6       0.19      0.23      0.21        13
          7       0.67      0.89      0.76         9
          8       0.84      0.84      0.84        19
          9       0.90      0.60      0.72        15

avg / total       0.62      0.61      0.60       150

Confusion Matrix


In [17]:
#  a probability based stack layer training set is used and stack layer classifier is DT
my_super_learner = SuperLearnerClassifier(base_model_list, stack_key='DT', proba_flag=True, original_data_flag = False)
my_super_learner.fit(X_train, y_train)

SuperLearnerClassifier(base_models={'LR': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'RF': RandomFor...       presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)},
            original_data_flag=False, param_grid={}, proba_flag=True,
            stack_key='DT')

In [18]:
evaluator(my_super_learner, X_test, y_test)

Accuracy: 0.64
             precision    recall  f1-score   support

          0       0.58      0.47      0.52        15
          1       0.89      0.94      0.92        18
          2       0.38      0.48      0.43        21
          3       0.60      0.55      0.57        11
          4       0.42      0.47      0.44        17
          5       0.83      0.83      0.83        12
          6       0.50      0.38      0.43        13
          7       0.58      0.78      0.67         9
          8       0.88      0.74      0.80        19
          9       0.86      0.80      0.83        15

avg / total       0.65      0.64      0.64       150

Confusion Matrix


In [20]:
#  a label based stack layer training set is used and stack layer classifier is LR
my_super_learner = SuperLearnerClassifier(base_model_list, stack_key='LR', proba_flag=False, original_data_flag = False)
my_super_learner.fit(X_train, y_train)

SuperLearnerClassifier(base_models={'LR': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'RF': RandomFor...       presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)},
            original_data_flag=False, param_grid={}, proba_flag=False,
            stack_key='LR')

In [21]:
evaluator(my_super_learner, X_test, y_test)

Accuracy: 0.3333333333333333
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        15
          1       0.54      0.72      0.62        18
          2       0.40      0.48      0.43        21
          3       0.12      0.09      0.11        11
          4       0.00      0.00      0.00        17
          5       0.37      0.58      0.45        12
          6       0.05      0.08      0.06        13
          7       0.36      0.56      0.43         9
          8       0.33      0.05      0.09        19
          9       0.43      0.80      0.56        15

avg / total       0.27      0.33      0.28       150

Confusion Matrix


In [22]:
#  a probability based stack layer training set is used and stack layer classifier is LR
my_super_learner = SuperLearnerClassifier(base_model_list, stack_key='LR', proba_flag=True, original_data_flag = False)
my_super_learner.fit(X_train, y_train)

SuperLearnerClassifier(base_models={'LR': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'RF': RandomFor...       presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)},
            original_data_flag=False, param_grid={}, proba_flag=True,
            stack_key='LR')

In [23]:
evaluator(my_super_learner, X_test, y_test)

Accuracy: 0.68
             precision    recall  f1-score   support

          0       0.50      0.47      0.48        15
          1       1.00      0.94      0.97        18
          2       0.48      0.62      0.54        21
          3       0.83      0.91      0.87        11
          4       0.50      0.29      0.37        17
          5       0.79      0.92      0.85        12
          6       0.20      0.23      0.21        13
          7       0.67      0.89      0.76         9
          8       0.95      0.95      0.95        19
          9       1.00      0.67      0.80        15

avg / total       0.70      0.68      0.68       150

Confusion Matrix


## Grid Search Through SuperLearnerClassifier Architectures & Parameters (Task 7)

Perfrom a grid search experiment to detemrine the optimal architecture and hyper-parameter values for the SuperLearnClasssifier for the MNIST Fashion classification problem.

Because the stacked ensemble model is based on the base estimators, so if the each of the base estimator has a better performance, the final result will be better to the second layer estimator. So I decide perform a grid search for each base estimators.

In [24]:
def GridSearchCV_Evaluator(model, param_grid, X, y, cv_folds = 3):
    # Perform the search
    my_tuned_model = GridSearchCV(model, param_grid, cv=cv_folds, verbose = 2, \
                            return_train_score=True)
    my_tuned_model.fit(X, y)
    
    return my_tuned_model


In [25]:
param_grid = {
    'proba_flag': [True, False],\
    'param_grid':[{
       'DT' :{'criterion': ['gini', "entropy"], \
             'max_depth': list(range(3, 50, 3)), \
             'min_samples_split': [200]},\
       'LR' :{'multi_class': ['ovr'], \
             'C': [x / 10.0 for x in range(2, 21, 2)],\
             'solver':['liblinear'],\
              'max_iter':[1000]},\
       'NN' :{'hidden_layer_sizes': [(400), (400, 200), (400, 200, 100)],\
               'alpha': list(10.0 ** -np.arange(1, 7))},\
       'SVM' :{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],\
                     'C': [1, 10, 100, 1000]},\
       'RF' :{"n_estimators" : [9, 18, 27, 36, 45, 54, 63],\
           "max_depth" : [1, 5, 10, 15, 20, 25, 30],\
           "min_samples_leaf" : [1, 2, 4, 6, 8, 10]},\
       'KNN' :{"n_neighbors" : list(range(1,10)),\
                'weights' : ['uniform','distance'],\
                'algorithm' :['auto','ball_tree','kd_tree','brute'],\
                'leaf_size' :list(range(1,2))},\
        'GB' :{'n_estimators':range(20,81,10)}
    }]
}

In [26]:
my_super_learner = SuperLearnerClassifier(base_model_list)
GridSearchCV_Evaluator(my_super_learner, param_grid, X_train, y_train, cv_folds = 3)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] param_grid={'DT': {'criterion': ['gini', 'entropy'], 'max_depth': [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48], 'min_samples_split': [200]}, 'LR': {'multi_class': ['ovr'], 'C': [0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0], 'solver': ['liblinear'], 'max_iter': [1000]}, 'NN': {'hidden_layer_sizes': [400, (400, 200), (400, 200, 100)], 'alpha': [0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06]}, 'SVM': {'kernel': ['rbf'], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]}, 'RF': {'n_estimators': [9, 18, 27, 36, 45, 54, 63], 'max_depth': [1, 5, 10, 15, 20, 25, 30], 'min_samples_leaf': [1, 2, 4, 6, 8, 10]}, 'KNN': {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'leaf_size': [1]}, 'GB': {'n_estimators': range(20, 81, 10)}}, proba_flag=True 
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=0.2, max_iter=1000, m

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  C=0.2, max_iter=1000, multi_class=ovr, solver=liblinear, total=   0.1s
[CV] C=0.2, max_iter=1000, multi_class=ovr, solver=liblinear .........
[CV]  C=0.2, max_iter=1000, multi_class=ovr, solver=liblinear, total=   0.1s
[CV] C=0.4, max_iter=1000, multi_class=ovr, solver=liblinear .........
[CV]  C=0.4, max_iter=1000, multi_class=ovr, solver=liblinear, total=   0.1s
[CV] C=0.4, max_iter=1000, multi_class=ovr, solver=liblinear .........
[CV]  C=0.4, max_iter=1000, multi_class=ovr, solver=liblinear, total=   0.1s
[CV] C=0.4, max_iter=1000, multi_class=ovr, solver=liblinear .........
[CV]  C=0.4, max_iter=1000, multi_class=ovr, solver=liblinear, total=   0.0s
[CV] C=0.6, max_iter=1000, multi_class=ovr, solver=liblinear .........
[CV]  C=0.6, max_iter=1000, multi_class=ovr, solver=liblinear, total=   0.1s
[CV] C=0.6, max_iter=1000, multi_class=ovr, solver=liblinear .........
[CV]  C=0.6, max_iter=1000, multi_class=ovr, solver=liblinear, total=   0.1s
[CV] C=0.6, max_iter=1000, multi_cl

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    4.6s finished


LR
Best parameters set found on development set:
{'C': 0.4, 'max_iter': 1000, 'multi_class': 'ovr', 'solver': 'liblinear'}
0.70995670995671
{'mean_fit_time': array([0.17580303, 0.15171234, 0.18289804, 0.1483593 , 0.14821021,
       0.15752276, 0.14316551, 0.12037905, 0.13676389, 0.13798571]), 'std_fit_time': array([0.03058975, 0.00587647, 0.01890091, 0.00120918, 0.01286072,
       0.01436151, 0.00978627, 0.00483604, 0.00881619, 0.01429003]), 'mean_score_time': array([0.00166623, 0.00166965, 0.00166766, 0.00133411, 0.00150363,
       0.00200073, 0.00200097, 0.00100271, 0.00116769, 0.00166782]), 'std_score_time': array([4.71820282e-04, 4.73231071e-04, 9.43471378e-04, 4.69909626e-04,
       4.12214086e-04, 8.15853522e-04, 8.15658924e-04, 7.37000982e-07,
       2.35573509e-04, 4.71763975e-04]), 'param_C': masked_array(data=[0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       f

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s


[CV] .................................. n_estimators=20, total=   2.3s
[CV] n_estimators=20 .................................................
[CV] .................................. n_estimators=20, total=   2.4s
[CV] n_estimators=30 .................................................
[CV] .................................. n_estimators=30, total=   4.4s
[CV] n_estimators=30 .................................................
[CV] .................................. n_estimators=30, total=   3.6s
[CV] n_estimators=30 .................................................
[CV] .................................. n_estimators=30, total=   3.6s
[CV] n_estimators=40 .................................................
[CV] .................................. n_estimators=40, total=   5.0s
[CV] n_estimators=40 .................................................
[CV] .................................. n_estimators=40, total=   5.2s
[CV] n_estimators=40 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:  2.0min finished


GB
Best parameters set found on development set:
{'n_estimators': 50}
0.7142857142857143
{'mean_fit_time': array([2.45514862, 3.94390583, 5.09114138, 6.14792546, 7.25296235,
       7.56838775, 8.16093508]), 'std_fit_time': array([0.02174837, 0.39843665, 0.14969935, 0.14680341, 0.21446177,
       0.27055567, 0.41005827]), 'mean_score_time': array([0.00150212, 0.00172774, 0.00245349, 0.00339707, 0.00246302,
       0.00283655, 0.00383647]), 'std_score_time': array([0.00040754, 0.00051882, 0.00064035, 0.00055275, 0.00052039,
       0.0006255 , 0.00062314]), 'param_n_estimators': masked_array(data=[20, 30, 40, 50, 60, 70, 80],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_estimators': 20}, {'n_estimators': 30}, {'n_estimators': 40}, {'n_estimators': 50}, {'n_estimators': 60}, {'n_estimators': 70}, {'n_estimators': 80}], 'split0_test_score': array([0.61344538, 0.66386555, 0.66386555, 0.68907563, 0.6806722

GridSearchCV(cv=3, error_score='raise',
       estimator=SuperLearnerClassifier(base_models={'LR': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, w...,
            original_data_flag=False, param_grid={}, proba_flag=False,
            stack_key='DT'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'proba_flag': [True, False], 'param_grid': [{'DT': {'criterion': ['gini', 'entropy'], 'max_depth': [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48], 'min_samples_split': [200]}, 'LR': {'multi_class': ['ovr'], 'C': [0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0], 'solver':...', 'ball_tree', 'kd_tree', 'brute'], 'leaf_size': [1]}, 'GB': {'n_estimators': range(20, 81, 10)}}]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, ve

Evaluate the performance of the model selected by the grid search on a hold-out dataset

In [19]:
evaluator(my_super_learner, X_test, y_test)

Accuracy: 0.8285714285714286
             precision    recall  f1-score   support

          0       0.86      0.66      0.75        29
          1       0.97      0.85      0.90        33
          2       0.91      0.73      0.81        41
          3       0.96      0.67      0.79        36
          4       0.65      0.50      0.57        26
          5       0.86      0.98      0.92        45
          6       0.52      0.95      0.67        39
          7       0.95      1.00      0.97        37
          8       0.94      0.94      0.94        35
          9       1.00      0.86      0.93        29

avg / total       0.86      0.83      0.83       350

Confusion Matrix


## Evaluating the Impact of Adding Original Descriptive Features at the Stack Layer (Task 8)

Evaluate the impact of adding original descriptive features at the stack layer.

In [27]:
turned_base_models = {
            'LR':linear_model.LogisticRegression(C = 0.2, max_iter = 1000, multi_class = 'ovr', solver = 'liblinear'), # Logistic Regression Classifier 
            'RF':ensemble.RandomForestClassifier(max_depth = 25, min_samples_leaf = 1, n_estimators = 18),  # Random Forest Classifier 
            'SVM':svm.SVC(C = 1, gamma = 0.001, kernel = 'rbf'),  # SVM Classifier
            'DT':tree.DecisionTreeClassifier(criterion = 'gini', max_depth = 3, min_samples_split = 200),  # Decision Tree Classifier
            'KNN':neighbors.KNeighborsClassifier(algorithm = 'auto', leaf_size = 1, n_neighbors = 8, weights = 'distance'), # KNN Classifier
            'NN':neural_network.MLPClassifier(alpha = 0.001, hidden_layer_sizes = (400, 200, 100)),  # Neural Network Classifier
            'GB':ensemble.GradientBoostingClassifier(n_estimators=50), # GradientBoostingClassifier 
        }

In [28]:
my_super_learner = SuperLearnerClassifier(turned_base_models, stack_key='DT', proba_flag=False, original_data_flag = True)
my_super_learner.fit(X_train, y_train)

SuperLearnerClassifier(base_models={'LR': LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'RF': RandomFo...       presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)},
            original_data_flag=True, param_grid={}, proba_flag=False,
            stack_key='DT')

In [30]:
evaluator(my_super_learner, X_test, y_test)

Accuracy: 0.5866666666666667
             precision    recall  f1-score   support

          0       0.60      0.40      0.48        15
          1       0.77      0.94      0.85        18
          2       0.44      0.71      0.55        21
          3       0.80      0.36      0.50        11
          4       0.44      0.24      0.31        17
          5       0.70      0.58      0.64        12
          6       0.21      0.31      0.25        13
          7       0.54      0.78      0.64         9
          8       0.87      0.68      0.76        19
          9       0.85      0.73      0.79        15

avg / total       0.62      0.59      0.58       150

Confusion Matrix


In [31]:
my_super_learner = SuperLearnerClassifier(turned_base_models, stack_key='DT', proba_flag=False, original_data_flag = False)
my_super_learner.fit(X_train, y_train)

SuperLearnerClassifier(base_models={'LR': LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'RF': RandomFo...       presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)},
            original_data_flag=False, param_grid={}, proba_flag=False,
            stack_key='DT')

In [32]:
evaluator(my_super_learner, X_test, y_test)

Accuracy: 0.6533333333333333
             precision    recall  f1-score   support

          0       0.41      0.47      0.44        15
          1       0.89      0.89      0.89        18
          2       0.45      0.67      0.54        21
          3       1.00      0.82      0.90        11
          4       0.43      0.18      0.25        17
          5       0.71      0.83      0.77        12
          6       0.38      0.38      0.38        13
          7       0.67      0.89      0.76         9
          8       0.89      0.84      0.86        19
          9       0.91      0.67      0.77        15

avg / total       0.67      0.65      0.65       150

Confusion Matrix


## Explore the Ensemble Model (Task 9)

Perform an analysis to investigate the strength of the base estimators and the strengths of the correlations between them.

Reference:

Combining Pattern Classifiers: Methods and Algorithms, Ludmila Kuncheva, 2004

Conditional Accuracy Table as in Combining Information Extraction Systems Using Voting and Stacked Generalization, Sigletos et al, 2005

Ensemble methods Foundations and Algorithms. Boca Raton, FL: Taylor & Francis. Zhou, Z. (2012).

In [33]:
class EnsembleDiversity(object):

    # predictions: list of lists. Each sublist contains the predictions of a classifier
    # names: list of strings. Each string is the name of the classifier.
    # true: list of labels. Each label is the truth label
    def __init__(self, predictions, names, true):

        N = len(true)
        labels = set(true)
        if len(predictions) != len(names):
            raise AttributeError('Number of classifiers is different than number \
                                  of names. %d != %d.' % (len(predictions), len(names)))
        for i, predict in enumerate(predictions):
            if len(predict) != N:
                raise AttributeError('Number of predictions of classifier %s is different\
                                      then the number of true labels. %d != %d .\
                                      ' % (names[i], len(predict), N))
            if labels.isdisjoint(set(predict)):
                raise AttributeError('Label in predictions of %s not in truth set. \
                                      ' % (names[i]))
        self.predictions = predictions
        self.names = names
        self.true = true

    def print_measures_details(self):

        """Print details about the computed measures"""

        print('===============================================================')
        print('Correlation: For +-1 perfect aggrement/disagreement')
        print('Q-statistic: Q=0  => Independent. For q>0 predictors find the the same results')
        print("Cohen's k: k->0  => High Disagreement => High Diversity" )
        print("Kohovi-Wolpert Variance -> Inf => High Diversity")
        print("Conditional Accuracy Table: Conditional Probability that the row system predicts correctly, given")
        print("                            that the column system also predicts correctly") 
        print('===============================================================')

        return

    def get_KWVariance(self, print_flag=True):

        """Get Kohovi Wolpert Variance."""

        kw = KW_Variance(self.predictions, self.true)
        if print_flag:
            print('#####  Kohovi-Wolpert Variance: % 0.3f  #####' % kw)
        return kw

    def get_avg_pairwise(self, print_flag=True):

        """Get average over all pairs in collection for different measures."""

        avg_metrics = avg_pairwise(self.predictions, self.names, self.true)
        if print_flag:
            print('#### Pairwise Average Metrics: #####')
            print('Avg. Cor: %0.3f' % avg_metrics[0])
            print('Avg. Q-statistic: %0.3f' % avg_metrics[1]) 
            print("Avg. Cohen's k: %0.3f" % avg_metrics[2]) 
        return avg_metrics

    def get_conditional_acc_table(self, print_flag=True):

        """Get the Conditional Accuracy Matric."""

        df_table = acc_cont_table(self.predictions, self.names, self.true, False)
        if print_flag:
            print('###Conditional Accuracy Table###')
            print(df_table.astype('float').to_string(float_format=lambda x: '%0.2f' % (x))) 
        return df_table

    def print_report(self):

        print( '---------------------------------------------------------------')
        print( 'Diversity Tests Report')
        print( '---------------------------------------------------------------\n')
        print( 'Measures Details')
        self.print_measures_details()
        print('---------------------------------------------------------------\n') 
        print('Measures Results') 
        print('---------------------------------------------------------------\n') 
        methods = [method for method in dir(self) if callable(getattr(self, method)) and 'get_' in method]
        for method in methods:
            method_run = getattr(self, method)
            method_run()
            print( '- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - \n')
        return


def Pairwise_Tests(y_a, y_b, y_true, name_1, name_2, p=0.05, print_cont = False):
    
    from numpy import zeros
    from pandas import DataFrame
    from scipy.stats import chi2
    from math import sqrt

    tresh = chi2.ppf(q=1 - p,  # Find the critical value for 95% confidence*
                     df=1)   # Df = number of variable categories - 1

    table_scores = zeros([2, 2], dtype=float)  # 2x2 McNemar Oracle Table
    counts_a = 0  # correct classifier a predictions
    counts_b = 0  # correct classifier b predictions
    for i, y in enumerate(y_true):
        if y == y_a[i]:
            counts_a += 1
            if y == y_b[i]:
                counts_b += 1
                table_scores[0, 0] += 1  # both a and b correct
            else:
                table_scores[0, 1] += 1  # a correct, b wrong
        else:
            if y == y_b[i]:
                counts_b += 1
                table_scores[1, 0] += 1  # b wrong, a correct
            else:
                table_scores[1, 1] += 1  # a, b both wrong
    b_given_a = table_scores[0, 0] / float(counts_a)
    a_given_b = table_scores[0, 0] / float(counts_b)
    if table_scores[0, 1] + table_scores[1, 0] == 0:  # can't find chi2
        chi_squared_value = 0
    else:  # calcualate chi2 value if possible
        chi_squared_value = pow(abs(table_scores[0, 1] - table_scores[1, 0]) - 1, 2) / float(table_scores[0, 1] + table_scores[1, 0])
    # Correlation between predictions
    correlation = (table_scores[0,0]*table_scores[1,1]-table_scores[0,1]*table_scores[1,0])/float(sqrt((table_scores[0,0]*table_scores[0,1])*(table_scores[1,0]*table_scores[1,1])*(table_scores[0,0]*table_scores[1,0])*(table_scores[0,1]*table_scores[1,1]) ))
    # Q statistic
    q_statistic = (table_scores[0,0]*table_scores[1,1]-table_scores[0,1]*table_scores[1,0])/float(table_scores[0,0]*table_scores[1,1]+table_scores[0,1]*table_scores[1,0])
    # Interater Agreement of predictions
    cohens_k = (table_scores[0,0]*table_scores[1,0]-table_scores[0,1]*table_scores[1,1])/float((table_scores[0,0]*table_scores[0,1])*(table_scores[1,0]+table_scores[1,1])+(table_scores[0,0]+table_scores[1,0])*(table_scores[0,1] +table_scores[1,1]))

    # Turn absolute counts in percentages for McNemar Table printing
    table_scores /= float(len(y_true))
    table_scores *= 100
    # Check for statistical independece according to chi2 test
    if chi_squared_value > tresh: 
        '!!! p=%0.2f Significant Difference: chi2-value %0.3f > %0.3f !!!' %(p, chi_squared_value, tresh)
    if print_cont: # print McNemar Table test
        print('Contigency Table: %s -%s' %(name_1, name_2)) 
        print('=====================') 
        df = DataFrame(table_scores, [name_1+'-cor', name_1+'-wro'], [name_2+'-cor', name_2+'-wro'])
        print(df.astype('float').to_string(float_format= lambda x: '%0.2f'%(x)))
        print('=====================') 
    return correlation, q_statistic, cohens_k, a_given_b, b_given_a


def KW_Variance(predictions, y_true):
    
    """Modification by Kuncheva et al. Expects a list of lists, containing predictions and a ground truth list."""

    correct_count = [0 for i in range(len(y_true))] # initialize correct counts per sample
    final_score = 0 # score to return
    for i, y in enumerate(y_true): # for each sample
        for pred in predictions: # check for each classifier
            if y == list(pred)[i]:
                correct_count[i] += 1 # if the prediction was correct, update counter
        final_score += correct_count[i]*(len(predictions)-correct_count[i]) # sum over all classifiers for this count

    return final_score/float(len(y_true)*pow(len(predictions),2))
                
    
def avg_pairwise(predictions, names, true):
    
    "Average of the pairwise metrics defined in PairWise Tests"
    
    avg_metrics = [0,0,0] # Differente Metrics Initialization
    num_pairs = 0 # Normalization factor
    for i in range(len(predictions)):
        for j in range(i+1, len(predictions)): # for each pair
            num_pairs +=1
            cor, q, k, _, _ = Pairwise_Tests(predictions[i], predictions[j], true, names[i], names[j])
            if type(cor) is float: # check to avoid when cor is infinite
                avg_metrics[0] += float(cor)
            avg_metrics[1] += q
            avg_metrics[2] += k
    avg_metrics = [avg/float(num_pairs) for avg in avg_metrics] # normalize over all pairs
    return avg_metrics    

def acc_cont_table(predictions, names, true, print_flag=True):
    
    from numpy import eye
    from pandas import DataFrame
    
    table = eye(len(predictions)) # table initilization
    for i in range(len(predictions)):
        for j in range(i+1, len(predictions)): # for each pair
            _, _, _, i_given_j, j_given_i = Pairwise_Tests(predictions[i], predictions[j], true, names[i], names[j])
            table[i, j] = i_given_j
            table[j, i] = j_given_i
    df = DataFrame(table, names, names)
    if print_flag:
        print(df.astype('float').to_string(float_format= lambda x: '%0.2f'%(x)))
    return df

In [34]:
my_super_learner = SuperLearnerClassifier(turned_base_models, stack_key='DT', proba_flag=False, original_data_flag = False)
my_super_learner.fit(X_train, y_train)

SuperLearnerClassifier(base_models={'LR': LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'RF': RandomFo...       presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)},
            original_data_flag=False, param_grid={}, proba_flag=False,
            stack_key='DT')

In [35]:
base_models = my_super_learner.base_models
predictions_test = my_super_learner.preds_list

test_class = EnsembleDiversity(predictions_test, list(base_models.keys()), y_train)
test_class.print_report()

---------------------------------------------------------------
Diversity Tests Report
---------------------------------------------------------------

Measures Details
Correlation: For +-1 perfect aggrement/disagreement
Q-statistic: Q=0  => Independent. For q>0 predictors find the the same results
Cohen's k: k->0  => High Disagreement => High Diversity
Kohovi-Wolpert Variance -> Inf => High Diversity
Conditional Accuracy Table: Conditional Probability that the row system predicts correctly, given
                            that the column system also predicts correctly
---------------------------------------------------------------

Measures Results
---------------------------------------------------------------

#####  Kohovi-Wolpert Variance:  0.178  #####
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

#### Pairwise Average Metrics: #####
Avg. Cor: 0.000
Avg. Q-statistic: 0.473
Avg. Cohen's k: 0.010
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

###Con

From the result report, we can see that the collection of the base estimators have high similarity and low Diversity,and that's the reason why the ensemble model's accuracy is not very high. So the basemodel list should be improved later.